In [2]:
import pandas as pd
import numpy as np
import lxml.html as lh
import requests
from bs4 import BeautifulSoup

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [4]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
[len(C) for (title,C) in col]

[288, 288, 288]

In [7]:
#create a dataframe by first collecting the data in a dictionary
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


In [8]:
#drop boroughs with value 'Not assigned'
df =df[df['Borough']!='Not assigned']

In [9]:
#delete unwantwed characters "/n" from neighbourhoods
df.astype('str')
df['Neighbourhood\n'] = df['Neighbourhood\n'].str[:-1]
df.columns=['borough', 'neighbourhood','postalcode']
df.head()

,borough,neighbourhood,postalcode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A


In [13]:
# combine names of neighbourhoods in same postal code
Add_hoods = df.groupby('postalcode')['neighbourhood'].apply(lambda x: "%s" % ', '.join(x))

<class 'pandas.core.frame.DataFrame'>


borough          object
neighbourhood    object
postalcode       object
dtype: object

In [44]:
#create new dataframe for further operations
df2 = pd.DataFrame(Add_hoods)
df2.reset_index(inplace = True)
df2.shape

(103, 2)

In [50]:
#create a dataframe with borough names and unique postal codes
boroughs = df.drop('neighbourhood', axis = 1)
boroughs = boroughs.drop_duplicates()
boroughs.reset_index(inplace = True)
boroughs.shape

(103, 1)

In [59]:
#add borough column to the dataframe with postal_codes and neigbourhoods
df2['borough'] = boroughs['borough']

In [72]:
cols = ['postalcode', 'borough', 'neighbourhood']
toronto_postal_codes = df2[cols]
toronto_postal_codes.head()

,postalcode,borough,neighbourhood
0,M1B,North York,"Rouge, Malvern"
1,M1C,North York,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Downtown Toronto,"Guildwood, Morningside, West Hill"
3,M1G,North York,Woburn
4,M1H,Queen's Park,Cedarbrae


In [73]:
toronto_postal_codes.shape

(103, 3)

In [103]:
body = client_258814a3521c4dad9efae98f049b719a.get_object(Bucket='dataanalysiswithpython-donotdelete-pr-yzsz0t5smbce1i',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

coordinates = pd.read_csv(body)
coordinates.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [105]:
coordinates.shape

(103, 3)

In [117]:
#toronto_postal_codes['latitude'] = coordinates['Latitude']
toronto_postal_codes['longitude'] = coordinates['Longitude']

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [118]:
toronto_postal_codes.head()

,postalcode,borough,neighbourhood,latitude,longitude
0,M1B,North York,"Rouge, Malvern",43.806686,-79.194353
1,M1C,North York,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Downtown Toronto,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,North York,Woburn,43.770992,-79.216917
4,M1H,Queen's Park,Cedarbrae,43.773136,-79.239476


In [119]:
import json # library to handle JSON files
!conda install -c conda-forge folium=0.5.0 --yes
import folium
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [121]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_postal_codes['borough'].unique()),
        toronto_postal_codes.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto

In [124]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="CN_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [126]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_postal_codes['latitude'], toronto_postal_codes['longitude'], toronto_postal_codes['borough'], toronto_postal_codes['neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [138]:
CLIENT_ID = 'IPSJ32SOFCJH4QI3ZHEWHSCP2CXO3UL0OYL1YU2DR0SK0YOA' # your Foursquare ID
CLIENT_SECRET = 'ZVDKLTJ1B5OPGJZHJWNQP4W5DPP2LCME0MH5VKOSMOQMAV5T' # your Foursquare Secret
VERSION = '20191605' # Foursquare API version
LIMIT = 30

## Explore Neighbourhoods in Toronto

#### create a data frame with only boroughs containing Toronto

In [134]:
toronto_data = toronto_postal_codes[toronto_postal_codes['borough'].str.contains('toronto', case = False)].reset_index(drop=True)
toronto_data.head()

,postalcode,borough,neighbourhood,latitude,longitude
0,M1E,Downtown Toronto,"Guildwood, Morningside, West Hill",43.763573,-79.188711
1,M1N,Downtown Toronto,"Birch Cliff, Cliffside West",43.692657,-79.264848
2,M1W,Downtown Toronto,L'Amoreaux West,43.799525,-79.318389
3,M2K,East Toronto,Bayview Village,43.786947,-79.385975
4,M2L,Downtown Toronto,"Silver Hills, York Mills",43.757490,-79.374714


#### Create a function to get venue info on all the neighborhoods in Toronto

In [131]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighbourhood', 
                  'neighbourhood latitude', 
                  'neighbourhood longitude', 
                  'venue', 
                  'venue latitude', 
                  'venue longitude', 
                  'venue category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [139]:
toronto_venues = getNearbyVenues(names=toronto_data['neighbourhood'],
                                   latitudes=toronto_data['latitude'],
                                   longitudes=toronto_data['longitude']
                                  )

Guildwood, Morningside, West Hill
Birch Cliff, Cliffside West
L'Amoreaux West
Bayview Village
Silver Hills, York Mills
Willowdale West
Parkwoods
CFB Toronto, Downsview East
Downsview West
Woodbine Heights
The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Davisville
Moore Park, Summerhill East
Ryerson, Garden District
Commerce Court, Victoria Hotel
Bedford Park, Lawrence Manor East
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
Humewood-Cedarvale
Caledonia-Fairbanks
Christie
Downsview, North Park, Upwood Park
Del Ray, Keelesdale, Mount Dennis, Silverthorn
The Junction North, Runnymede
Parkdale, Roncesvalles
Runnymede, Swansea
Canada Post Gateway Processing Centre
Business Reply Mail Processing Centre 969 Eastern
Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Roya

#### Check the size of the resulting dataframe

In [141]:
print(toronto_venues.shape)
toronto_venues.head()

(469, 7)


,neighbourhood,neighbourhood latitude,neighbourhood longitude,venue,venue latitude,venue longitude,venue category
0,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
1,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


#### Let's check how many venues were returned for each neighborhood

In [142]:
toronto_venues.groupby('neighbourhood').count()

,neighbourhood latitude,neighbourhood longitude,venue,venue latitude,venue longitude,venue category
neighbourhood,,,,,,
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CFB Toronto, Downsview East",3,3,3,3,3,3
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
Caledonia-Fairbanks,6,6,6,6,6,6
Canada Post Gateway Processing Centre,10,10,10,10,10,10
"Chinatown, Grange Park, Kensington Market",30,30,30,30,30,30


#### How many unique categories can be curated from all the returned venues?

In [145]:
print('There are {} uniques categories.'.format(len(toronto_venues['venue category'].unique())))

There are 160 uniques categories.


## Analyze Each Neighborhood

In [174]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['venue category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['neighbourhood'] = toronto_venues['neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,...,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [175]:
#examine dataframe size
toronto_onehot.shape

(469, 161)

In [176]:
toronto_grouped = toronto_onehot.groupby('neighbourhood').mean().reset_index()
toronto_grouped

,neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,...,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.043478,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
4,"CFB Toronto, Downsview East",0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Caledonia-Fairbanks,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000
7,Canada Post Gateway Processing Centre,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.100000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.033333,0.000000,0.000000,0.066667,0.066667,0.033333,0.000000,0.000000,0.000000
9,Christie,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Print each neighborhood along with the top 5 most common venues

In [177]:
num_top_venues = 5

for hood in toronto_grouped['neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4    Mobile Phone Shop  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0       Italian Restaurant  0.09
1     Fast Food Restaurant  0.09
2              Coffee Shop  0.09
3         Greek Restaurant  0.04
4  Comfort Food Restaurant  0.04


----Birch Cliff, Cliffside West----
                   venue  freq
0                   Café  0.25
1           Skating Rink  0.25
2        College Stadium  0.25
3  General Entertainment  0.25
4                Airport  0.00


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2                Park  0.06
3          Comic Shop  0.06
4    Recording Studio  0.06


----CFB Toronto, Downsview East----
                      venue  freq
0                   Airport  0.33
1 

#### Put that into a pandas dataframe

create a function to sort the venues in descending order.

In [184]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood

In [185]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['neighbourhood'] = toronto_grouped['neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Yoga Studio,Diner,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
1,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Sushi Restaurant,Pizza Place,Restaurant,Breakfast Spot,Juice Bar,Butcher,Pub
2,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Smoke Shop,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place,Recording Studio
4,"CFB Toronto, Downsview East",Airport,Park,Snack Place,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 4 clusters.

In [186]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 0, 1, 1, 1], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [187]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('neighbourhood'), on='neighbourhood')

toronto_merged.head() # check the last columns!

,postalcode,borough,neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1E,Downtown Toronto,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Rental Car Location,Mexican Restaurant,Breakfast Spot,Spa,Medical Center,Intersection,Electronics Store,Pizza Place,Creperie,Cuban Restaurant
1,M1N,Downtown Toronto,"Birch Cliff, Cliffside West",43.692657,-79.264848,1,College Stadium,General Entertainment,Skating Rink,Café,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
2,M1W,Downtown Toronto,L'Amoreaux West,43.799525,-79.318389,1,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Pharmacy,Japanese Restaurant,Sandwich Place,Breakfast Spot,Electronics Store,Coffee Shop,Pizza Place
3,M2K,East Toronto,Bayview Village,43.786947,-79.385975,1,Chinese Restaurant,Bank,Japanese Restaurant,Café,Yoga Studio,Diner,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
4,M2L,Downtown Toronto,"Silver Hills, York Mills",43.757490,-79.374714,0,Park,Cafeteria,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner


In [188]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine and name clusters

#### Cluster 1 - parks and airports

In [189]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,0,Park,Cafeteria,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
6,Downtown Toronto,0,Park,Fast Food Restaurant,Pool,Food & Drink Shop,Yoga Studio,Deli / Bodega,Dumpling Restaurant,Dog Run,Discount Store,Diner
7,Downtown Toronto,0,Airport,Park,Snack Place,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
23,Central Toronto,0,Park,Pharmacy,Women's Store,Fast Food Restaurant,Market,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
25,Central Toronto,0,Park,Construction & Landscaping,Massage Studio,Bakery,Yoga Studio,Diner,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
37,East Toronto,0,Pizza Place,Park,Mobile Phone Shop,Bus Line,Deli / Bodega,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


#### Cluster 2 - miscellaneous

In [190]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Rental Car Location,Mexican Restaurant,Breakfast Spot,Spa,Medical Center,Intersection,Electronics Store,Pizza Place,Creperie,Cuban Restaurant
1,Downtown Toronto,1,College Stadium,General Entertainment,Skating Rink,Café,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
2,Downtown Toronto,1,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Pharmacy,Japanese Restaurant,Sandwich Place,Breakfast Spot,Electronics Store,Coffee Shop,Pizza Place
3,East Toronto,1,Chinese Restaurant,Bank,Japanese Restaurant,Café,Yoga Studio,Diner,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
5,Downtown Toronto,1,Pizza Place,Butcher,Coffee Shop,Pharmacy,Grocery Store,Garden Center,Cuban Restaurant,Dog Run,Discount Store,Diner
8,West Toronto,1,Moving Target,Grocery Store,Shopping Mall,Bank,Yoga Studio,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
9,Downtown Toronto,1,Pharmacy,Cosmetics Shop,Skating Rink,Beer Store,Asian Restaurant,Curling Ice,Park,Convenience Store,Coworking Space,Creperie
10,West Toronto,1,Pub,Neighborhood,Health Food Store,Music Venue,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
11,East Toronto,1,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Indian Restaurant,Cosmetics Shop,Coffee Shop,Pizza Place,Dessert Shop,Pub
12,Downtown Toronto,1,Park,Sandwich Place,Ice Cream Shop,Fast Food Restaurant,Liquor Store,Brewery,Light Rail Station,Burrito Place,Italian Restaurant,Pub


#### Cluster 3 - concert hall

In [191]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Downtown Toronto,2,Pizza Place,Empanada Restaurant,Comic Shop,Concert Hall,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner


#### Cluster 4 - baseball fields

In [192]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Downtown Toronto,3,Baseball Field,Yoga Studio,Farmers Market,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
35,Downtown Toronto,3,Construction & Landscaping,Baseball Field,Yoga Studio,Diner,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
